In [1]:
#Import Dependencies

from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np

# Imports for routes
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [2]:
# Sets a path to Google Chrome
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/joescuteri/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [3]:
# URL of the page to be scraped
url = 'https://twitchtracker.com/statistics/games'
# Visit the URL
browser.visit(url)
# Parse the link using BeautifulSoup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Find the games and their respective views from page 1 of the table
games1 = []
views1 = []

table1 = soup.find('table', id='share-table')

for i in table1.find_all('a'):
    output = i.text
    games1.append(output)

for i in table1.find_all('td', class_="sorting_1"):
    output = i.text
    views1.append(output)

In [14]:
# # Navigate to page two
# page_two = browser.links.find_by_partial_text('2')[1].click()

In [16]:
# Find the games and their respective views from page 1 of the table

# games2 = []
# views2 = []

#table2 = soup.find('table', id='share-table')

# for i in table2.find_all('a'):
#     output = i.text
#     games2.append(output)
    
# for i in table1.find_all('td', class_="sorting_1"):
#     output = i.text
#     views2.append(output)

In [9]:
# Remove "All Other Games Combined" from views1
del views1[0]

# Remove leading and trailing spaces
views1cleaned = []

for i in views1:
    j = i.replace(' ','')
    views1cleaned.append(j)

In [10]:
# Upload to DataFrame and remove extra characters
df3 = pd.DataFrame(games1, columns = ['GAMES'])
df3['VIEWS'] = views1cleaned
df3 = df3.replace(r'\n+|\t+','', regex=True)

In [11]:
# Convert DataFrame into Dictionary
twitch = pd.Series(df3.VIEWS.values,index=df3.GAMES).to_dict()

In [12]:
twitch

{'Just Chatting': '374,034',
 'League of Legends': '269,528',
 'Rust': '223,036',
 'Grand Theft Auto V': '198,498',
 'Escape from Tarkov': '122,840',
 'VALORANT': '116,776',
 'Apex Legends': '96,378',
 'Fortnite': '89,392',
 'Dota 2': '71,811'}

# Migrating it to MongoDB

In [26]:
# connecting to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# declare the database
db = client.twitchviews_db
collection = db.items

In [29]:
# Push into MongoDB
collection.insert_one(twitch)